In [1]:
import pandas as pd

df = pd.read_csv('final.csv')

df.head()

,Unnamed: 0,Track,Album Name,Artist,Release Date,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,...,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,Explicit Track,Total_counts,Total_reach,Total_streams,Total_likes,Total_Views
0,0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,2024-04-26,1.0,725.4,3.904709e+08,30716.0,196631588.0,...,18004655.0,22931.0,4818457.0,2669262.0,0,2700154.0,3.648273e+08,413294048.0,6.532790e+08,5.416557e+09
1,1,Not Like Us,Not Like Us,Kendrick Lamar,2024-05-04,2.0,545.9,3.237039e+08,28113.0,174597137.0,...,7780028.0,28444.0,6623075.0,1118279.0,1,1146570.0,3.413999e+08,338106987.0,3.871029e+07,3.246861e+08
2,2,i like the way you kiss me,I like the way you kiss me,Artemas,2024-03-19,3.0,538.4,6.013093e+08,54331.0,211607669.0,...,5022621.0,5639.0,7208651.0,5285340.0,0,5339979.0,6.217145e+08,613540555.0,2.773830e+08,3.491720e+09
3,3,Flowers,Flowers - Single,Miley Cyrus,2023-01-12,4.0,444.9,2.031281e+09,269802.0,136569078.0,...,190260277.0,203384.0,NaN,11822942.0,0,12093218.0,3.512442e+09,756786414.5,1.089388e+09,1.569983e+10
4,4,Houdini,Houdini,Eminem,2024-05-31,5.0,423.3,1.070349e+08,7223.0,151469874.0,...,4493884.0,7006.0,207179.0,457017.0,1,464427.0,2.818943e+08,111735985.0,8.017811e+07,1.769638e+09


In [2]:
df.drop('Unnamed: 0' , axis=1, inplace=True)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4595 entries, 0 to 4594
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Track                       4595 non-null   object 
 1   Album Name                  4595 non-null   object 
 2   Artist                      4595 non-null   object 
 3   Release Date                4595 non-null   object 
 4   All Time Rank               4595 non-null   float64
 5   Track Score                 4595 non-null   float64
 6   Spotify Streams             4487 non-null   float64
 7   Spotify Playlist Count      4530 non-null   float64
 8   Spotify Playlist Reach      4528 non-null   float64
 9   Spotify Popularity          3796 non-null   float64
 10  YouTube Views               4292 non-null   float64
 11  YouTube Likes               4285 non-null   float64
 12  TikTok Posts                3427 non-null   float64
 13  TikTok Likes                3620 

In [48]:
X = df[['Spotify Popularity', 'All Time Rank', 'Amazon Playlist Count']]

y = df['Track Score']


In [57]:
print("Missing values in features before imputation:")
print(X.isna().sum())

print("\nMissing values in target before imputation:")
print(y.isna().sum())

Missing values in features before imputation:
Spotify Popularity        799
All Time Rank               0
Amazon Playlist Count    1050
dtype: int64

Missing values in target before imputation:
0


In [59]:
columns_to_fill = ['Spotify Popularity', 'All Time Rank', 'Amazon Playlist Count']
target_column = 'Track Score'

artist_averages = df.groupby('Artist')[columns_to_fill].mean()

def fill_missing_with_artist_avg(row, artist_averages):
    artist = row['Artist']
    if artist in artist_averages.index:
        for column in columns_to_fill:
            if pd.isna(row[column]):
                row[column] = artist_averages.loc[artist, column]
    return row

df = df.apply(lambda row: fill_missing_with_artist_avg(row, artist_averages), axis=1)

df_clean = df.dropna(subset=[target_column])

X_clean = df_clean[columns_to_fill]
y_clean = df_clean[target_column]

print(f"\nFeatures shape after filling missing values: {X_clean.shape}")
print(f"Target shape after handling missing values: {y_clean.shape}")

print("\nMissing values in features after filling with artist averages:")
print(X_clean.isna().sum())
print("\nMissing values in target after handling:")
print(y_clean.isna().sum())


Features shape after filling missing values: (4595, 3)
Target shape after handling missing values: (4595,)

Missing values in features after filling with artist averages:
Spotify Popularity       361
All Time Rank              0
Amazon Playlist Count    707
dtype: int64

Missing values in target after handling:
0


In [62]:

columns_to_check = ['Spotify Popularity', 'All Time Rank', 'Amazon Playlist Count', 'Track Score']

df_clean = df.dropna(subset=columns_to_check)

X_clean = df_clean[['Spotify Popularity', 'All Time Rank', 'Amazon Playlist Count']]
y_clean = df_clean['Track Score']

print(f"\nFeatures shape after dropping NaNs: {X_clean.shape}")
print(f"Target shape after dropping NaNs: {y_clean.shape}")

print("\nMissing values in features after dropping NaNs:")
print(X_clean.isna().sum())
print("\nMissing values in target after dropping NaNs:")
print(y_clean.isna().sum())



Features shape after dropping NaNs: (3786, 3)
Target shape after dropping NaNs: (3786,)

Missing values in features after dropping NaNs:
Spotify Popularity       0
All Time Rank            0
Amazon Playlist Count    0
dtype: int64

Missing values in target after dropping NaNs:
0


In [64]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X_clean)

In [65]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV


model1 = Ridge()
model2 = RandomForestRegressor()
model3 = GradientBoostingRegressor()

param_grid1 = {'alpha': [0.1, 1.0, 10.0]}

grid1 = GridSearchCV(model1, param_grid1, cv=10)

grid1.fit(X_scaled, y_clean)

best_model1 = grid1.best_estimator_

In [66]:
from sklearn.ensemble import VotingRegressor

voting_regressor = VotingRegressor(estimators=[
    ('ridge', best_model1),
    ('random_forest', model2),
    ('gradient_boosting', model3)
])

voting_regressor.fit(X_scaled, y_clean)


VotingRegressor(estimators=[('ridge', Ridge(alpha=10.0)),
                            ('random_forest', RandomForestRegressor()),
                            ('gradient_boosting', GradientBoostingRegressor())])

In [68]:
from sklearn.metrics import mean_squared_error , r2_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_clean, test_size=0.1, random_state=42)

voting_regressor.fit(X_train, y_train)

y_pred = voting_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')



Mean Squared Error: 69.64338540387398


In [69]:
r2 = r2_score(y_test, y_pred)
print(f'R^2 Score: {r2}')

R^2 Score: 0.9572630822811518


In [73]:
df.head(1)["Track Score"]

0    725.4
Name: Track Score, dtype: float64

In [ ]:
##'Spotify Popularity', 'All Time Rank', 'Amazon Playlist Count

In [74]:
adjustment_factor = 1.05
Track_Score = voting_regressor.predict([[92.0 , 2.0, 114]]) * adjustment_factor

In [75]:
Track_Score

array([584.81077917])

In [76]:
import joblib
filename = 'model.sav'
joblib.dump(voting_regressor, filename)

['model.sav']